In [23]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
import shap
import warnings
warnings.filterwarnings('ignore')

# Load cleaned data and prepared splits (reuse from notebook 3)
df = pd.read_csv('amazon_sales_cleaned.csv')
# Recreate features and splits as done in notebook 3 (code omitted for brevity, assume we have X_train, X_test, etc.)

# For demonstration, we focus on two models: Random Forest Regressor for total_revenue and XGBoost Classifier for high_value.


In [ ]:
# 1. Hyperparameter Tuning for Random Forest Regressor 

# Use RandomizedSearchCV for efficiency
param_dist_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
rand_search_rf = RandomizedSearchCV(rf_base, param_dist_rf, n_iter=10, cv=3, scoring='r2', random_state=42, n_jobs=-1)
rand_search_rf.fit(X_train, y_train_rev)   # assuming X_train is original (unscaled) from notebook 3

print("Best RF params:", rand_search_rf.best_params_)
best_rf = rand_search_rf.best_estimator_

# Evaluate
pred_rf = best_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test_rev, pred_rf))
r2_rf = r2_score(y_test_rev, pred_rf)
print(f"Tuned RF - RMSE: {rmse_rf:.2f}, R2: {r2_rf:.3f}")

In [22]:
# 2. Hyperparameter Tuning for XGBoost Classifier (high-value) 

param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_clf = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
rand_search_xgb = RandomizedSearchCV(xgb_clf, param_dist_xgb, n_iter=10, cv=3, scoring='accuracy', random_state=42, n_jobs=-1)
rand_search_xgb.fit(X_train_hv, y_train_hv)   # assuming X_train_hv from notebook 3

print("\nBest XGBoost params:", rand_search_xgb.best_params_)
best_xgb = rand_search_xgb.best_estimator_

# Evaluate
pred_xgb = best_xgb.predict(X_test_hv)
acc_xgb = accuracy_score(y_test_hv, pred_xgb)
print(f"Tuned XGBoost - Accuracy: {acc_xgb:.3f}")
print("\nClassification Report:\n", classification_report(y_test_hv, pred_xgb))


Best XGBoost params: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Tuned XGBoost - Accuracy: 0.999


NameError: name 'classification_report' is not defined